In this notebook, we'll show a bit behind the scenes about how to solve partial differential equations that have a positivity constraint.
We'll start with the simplest problem we can get our hands on -- advection of a scalar field $\phi$ in a constant velocity $u$.
The variational form of this problem is that, for all test functions $u$,
$$\int_\Omega\left(\partial_t\phi\cdot\psi - \phi u \cdot\nabla\psi - a\cdot\phi\right)\; dx = 0$$
for all scalar test functions $\psi$.
Here $a$ is the accumulation/ablation function.

The unusual feature that we'd like to show here is how to impose also an additional constraint $\phi \ge 0$.
Inequality constraints like this one show up in real applications very often.
For example, when simulating the flow of glaciers or surface water runoff, we require that the ice or water film thickness is positive.
There might be ablation (negative values of a) even where the solution is already 0 and, if not addressed at all, this will lead to unphysical negative values.
The blunt way of dealing with this problem is to clamp the solution from below, but this introduces a conservation error.
Instead, we can think of the PDE as a *complementarity problem* and use the appropriate methods.

In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm, trange
import firedrake
from firedrake import inner, grad, dx, ds, max_value, Constant
import irksome
from irksome import Dt

Most of the existing work on solving PDEs with positivity constraints assume that we're using low-order basis functions -- piecewise constant or linear.
Low-order basis functions are nice because we can tell if the function is positive just by looking at whether its coefficients are positive.

We'd like to try using piecewise quadratic or higher order basis functions.
However, if we use the usual Lagrange finite element basis, there is no relation between positivity of the coefficients in this basis and positivity of the function itself.
Instead, we'll use the basis of Bernstein polynomials.
If the coefficients of a function in the Bernstein basis are positive, then the function is positive.
The converse is not necessarily true -- there are positive functions that have some negative Bernstein coefficients -- but they eventually have all-positive coefficients on mesh refinement.

In [ ]:
num_levels = 5
mesh = firedrake.UnitDiskMesh(num_levels)
Q = firedrake.FunctionSpace(mesh, "Bernstein", 2)

For a velocity field, we'll use uniform solid-body rotation.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
u = firedrake.as_vector((-x[1], x[0]))

As initial data, we'll consider a spherical blip of radius 1/3 centered at the point (0.5, 0.0).

In [ ]:
ξ = Constant((0.5, 0.0))
r = Constant(1 / 3)
expr = max_value(0, 1 - inner(x - ξ, x - ξ) / r**2)
ϕ = firedrake.project(expr, Q)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(ϕ, axes=ax)
fig.colorbar(colors);

The sink term will be a spherical blob of radius 1/6 centered at the point (-0.5, 0).
We've chosen a rapid rate of ablation but a small radius so that part of the solution is zeroed out.

In [ ]:
a_0 = Constant(10.0)
ζ = Constant((-0.5, 0.0))
ρ = Constant(1 / 6)
expr = -a_0 * max_value(0, 1 - inner(x - ζ, x - ζ) / ρ**2)
a = firedrake.project(expr, Q)

We'll do a dirty hack here to make sure the forcing function is strictly negative.

In [ ]:
a.dat.data[:] = np.minimum(a.dat.data_ro, 0.0)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(a, axes=ax)
fig.colorbar(colors);

Create the variational form of the advection equation.

In [ ]:
ψ = firedrake.TestFunction(Q)
F = (Dt(ϕ) * ψ - inner(ϕ * u, grad(ψ)) - a * ψ) * dx

Now we'll create a timestepping scheme.
First, we need to pick the timestep and the number of steps to use.

In [ ]:
t = Constant(0.0)
num_steps = 512
dt = Constant(2 * π / num_steps)

Now the interesting part.
We first need to create two functions representing the upper and lower bounds the solution can take.

In [ ]:
lower = firedrake.Function(Q)
upper = firedrake.Function(Q)
upper.assign(+np.inf)
bounds = ("stage", lower, upper)

Then we need to tell Irksome that we're solving a bounds-constrained problem.
First, we need to use PETSc's [VINEWTONRSLS](https://petsc.org/release/manualpages/SNES/SNESVINEWTONRSLS/) solver.
It is the only PETSc solver that can handle nonlinear systems with inequality constraints.
We need to specify that we're using the stage-value form, rather than the stage-derivative form, of Runge Kutta methods.
We're using the Bernstein basis in time instead of the Lagrange basis.
Finally, we add the bounds constraints themselves.

In [ ]:
params = {
    "solver_parameters": {"snes_type": "vinewtonrsls"},
    "stage_type": "value",
    "basis_type": "Bernstein",
    "bounds": bounds,
}

Now we'll create a time stepper object.
Here we're using the Radau-IIA(2) scheme, which is 3rd-order in time.
The Radau-IIA(2) scheme is based on using a quadratic collocation polynomial in time.
Maintaining positivity with higher-order collocation polynomials is very new.

In [ ]:
method = irksome.RadauIIA(2)
solver = irksome.TimeStepper(F, method, t, dt, ϕ, **params)

Solve the problem forward in time.

In [ ]:
ϕs = [ϕ.copy(deepcopy=True)]

for step in trange(num_steps):
    solver.advance()
    t.assign(float(t) + float(dt))
    ϕs.append(ϕ.copy(deepcopy=True))

Make an animation of the solution.

In [ ]:
%%capture
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(
    ϕs[0], vmin=0.0, vmax=1.0, num_sample_points=4, axes=ax
)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(ϕ):
    colors.set_array(fn_plotter(ϕ))

In [ ]:
interval = 1e3 * 20 / num_steps
animation = FuncAnimation(fig, animate, frames=tqdm(ϕs), interval=interval)

In [ ]:
HTML(animation.to_html5_video())

Half-way through the simulation, the initial blob advects into the sink.
But by posing the problem as a variational inequality, we can make sure the solution does not go negative.
In the movie you might notice that the solution develops some oscillations, which is undesirable.
Later we'll show how to get rid of those by using a discontinuous Galerkin discretization.

To finish things off, let's look at the total mass in the system to see if there are any appreciable conservation errors.

In [ ]:
volumes = [firedrake.assemble(ϕ * dx) for ϕ in ϕs]
fig, ax = plt.subplots()
ax.plot(volumes);

We can see that there is a small spurious source -- a relative mass gain of less than 1\% -- before the solution hits the sink.

In [ ]:
(volumes[150] - volumes[0]) / volumes[0]